In [13]:
import random
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from io import StringIO
from urllib.parse import quote
import re
import unicodedata

In [14]:
def strip_accents(s):
    """Remove accents from a given string."""
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

def generate_morph_code(part_of_speech, person, number, tense, mood, voice, gender, case, degree):
    """
    Generate a morph code based on grammatical features according to the 9-position key.
    """
    morph_code = f"{part_of_speech}{person}{number}{tense}{mood}{voice}{gender}{case}{degree}"
    return morph_code

def normalize_word(word):
    """Normalize a Greek word by removing accents and converting to lowercase."""
    return strip_accents(word).lower()



In [15]:
def parse_noun_table(df, base_word, gender, sequence_num_start, definition):
    """Parse the noun inflection table and extract forms along with their grammatical features."""
    # Map for cases and numbers
    case_map = {
        'nominative': 'n',
        'genitive': 'g',
        'dative': 'd',
        'accusative': 'a',
        'vocative': 'v',
        'ablative': 'b',
        'locative': 'l',
        'instrumental': 'i',
        'notes:': ''
    }
    number_map = {
        'singular': 's',
        'dual': 'd',
        'plural': 'p'
    }

    exclude_texts = set([
        'adverb', 'comparative', 'superlative', '-', '—', '', ' ', 'notes:'
    ])

    forms = []
    sequence_num = sequence_num_start

    # Clean the DataFrame headers
    df.columns = [str(col).lower().strip() for col in df.columns]

    # The first column should be cases
    cases = df.iloc[:, 0].astype(str).str.lower().str.strip()
    # The rest of the columns are numbers
    numbers = [str(col).lower().strip() for col in df.columns[1:]]

    for idx, case in enumerate(cases):
        case = case.strip()
        case_key = case_map.get(case, '-')
        if case_key == '' or case in exclude_texts:
            continue  # Skip notes or irrelevant rows
        for col_idx, num_col in enumerate(numbers):
            num_col = num_col.strip()
            number_key = number_map.get(num_col, '-')
            if number_key == '-' or num_col in exclude_texts:
                continue
            form = df.iloc[idx, col_idx + 1]
            if isinstance(form, str):
                form = form.strip()
                if form != '' and form not in exclude_texts:
                    bare_text = strip_accents(form)
                    bare_base_form = strip_accents(base_word)
                    morph_code = generate_morph_code(
                        part_of_speech='n',
                        person='-',
                        number=number_key,
                        tense='-',
                        mood='-',
                        voice='-',
                        gender=gender,
                        case=case_key,
                        degree='-'
                    )
                    form_entry = {
                        'id': sequence_num,
                        'text': form,
                        'bare_text': bare_text,
                        'sequence_num': sequence_num,
                        'morph_code': morph_code,
                        'base_form': base_word,
                        'bare_base_form': bare_base_form,
                        'definition': definition
                    }
                    forms.append(form_entry)
                    sequence_num += 1
    return forms

def parse_adjective_table(df, base_word, sequence_num_start, definition):
    """
    Parse the adjective inflection table and extract forms along with their grammatical features.
    Handles tables with extra header rows and reconstructs columns accordingly.
    """
    # Map for degrees
    degree_map = {
        'positive': 'p',
        'comparative': 'c',
        'superlative': 's'
    }
    # Map for genders
    gender_map = {
        'masculine': 'm',
        'feminine': 'f',
        'masculine / feminine': 'mf',
        'masculine/feminine': 'mf',
        'neuter': 'n'
    }
    # Map for cases
    case_map = {
        'nominative': 'n',
        'genitive': 'g',
        'dative': 'd',
        'accusative': 'a',
        'vocative': 'v',
        'ablative': 'b',
        'locative': 'l',
        'instrumental': 'i',
        'notes:': ''
    }
    # Map for numbers
    number_map = {
        'singular': 's',
        'dual': 'd',
        'plural': 'p'
    }

        # Set of unwanted entries to exclude
    exclude_texts = set([
        'adverb', 'comparative', 'superlative', '-', '—', '', ' ', 'notes:'
    ])

    forms = []
    sequence_num = sequence_num_start

    # Adjust the DataFrame
    # Set columns to the first row
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    # Clean up column names
    df.columns = [str(col).strip().lower() for col in df.columns]
    # Drop columns with 'nan' names
    df = df.loc[:, df.columns != 'nan']

    # Reconstruct df.columns to include 'number' and 'gender'
    df_columns = []
    for idx, col in enumerate(df.columns):
        if idx == 0:
            # First column is 'case'
            df_columns.append('case')
        else:
            if idx in [1, 2]:
                number = 'singular'
            elif idx in [3, 4]:
                number = 'dual'
            elif idx in [5, 6]:
                number = 'plural'
            else:
                number = ''
            gender = col.strip().lower()
            df_columns.append(f"{number} {gender}")

    df.columns = df_columns

    # Extract cases
    cases = df['case'].str.lower().str.strip()

    for idx, case in enumerate(cases):
        case_key = case_map.get(case, '-')
        if case_key == '':
            continue  # Skip notes or irrelevant rows

        for col_idx in range(1, len(df.columns)):
            col_name = df.columns[col_idx]
            if any(ex_text in col_name for ex_text in exclude_texts):
                continue
            tokens = col_name.split()
            if len(tokens) >= 2:
                number_name = tokens[0]
                gender_name = ' '.join(tokens[1:])
            else:
                continue  # Cannot extract number and gender

            number_key = number_map.get(number_name.strip(), '-')
            gender_key = gender_map.get(gender_name.strip(), '-')
            if number_key == '-' or gender_key == '-':
                continue

            form = df.iloc[idx, col_idx]
            if isinstance(form, str):
                form = form.strip()
                if form != '':
                    bare_text = strip_accents(form)
                    bare_base_form = strip_accents(base_word)
                    # Assume degree is 'positive' unless specified
                    degree_key = 'p'
                    # Handle 'masculine / feminine' gender
                    if gender_key == 'mf':
                        genders = ['m', 'f']
                    else:
                        genders = [gender_key]
                    for gender in genders:
                        morph_code = generate_morph_code(
                            part_of_speech='a',
                            person='-',
                            number=number_key,
                            tense='-',
                            mood='-',
                            voice='-',
                            gender=gender,
                            case=case_key,
                            degree=degree_key
                        )
                        form_entry = {
                            'id': sequence_num,
                            'text': form,
                            'bare_text': bare_text,
                            'sequence_num': sequence_num,
                            'morph_code': morph_code,
                            'base_form': base_word,
                            'bare_base_form': bare_base_form,
                            'definition': definition
                        }
                        forms.append(form_entry)
                        sequence_num += 1

    return forms


def parse_verb_table(df, base_word, sequence_num_start, definition):
    """
    Parse the verb inflection table and extract forms along with their grammatical features.
    Handles tables with multiple header levels.
    """
    # Maps for grammatical features
    person_map = {
        'first person': '1',
        'second person': '2',
        'third person': '3',
    }
    
    number_map = {
        'singular': 's',
        'dual': 'd',
        'plural': 'p'
    }
    
    mood_map = {
        'indicative': 'i',
        'subjunctive': 's',
        'optative': 'o',
        'imperative': 'm',
        'infinitive': 'n',
        'participle': 'p'
    }
    
    voice_map = {
        'active': 'a',
        'middle': 'm',
        'passive': 'p',
        'mediopassive': 'e'
    }
    
    tense_map = {
        'present': 'p',
        'imperfect': 'i',
        'future': 'f',
        'aorist': 'a',
        'perfect': 'r',
        'pluperfect': 'l',
        'future perfect': 't'
    }
        # Set of unwanted entries to exclude
    exclude_texts = set([
        'adverb', 'comparative', 'superlative', '-', '—', '', ' ', 'notes:'
    ])

    forms = []
    sequence_num = sequence_num_start

    # Flatten MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [' '.join([str(s).strip().lower() for s in col if str(s) != 'nan']) for col in df.columns]
    else:
        df.columns = [col.lower().strip() for col in df.columns]

    # Identify the grammatical features from the table
    for col in df.columns:
        if ' ' in col:
            continue
        if col in ['person', 'number', 'person / number']:
            # This is the row index column
            df.rename(columns={col: 'person_number'}, inplace=True)
            break

    # Extract mood, voice, tense from the table caption or nearby headers
    # For this example, let's assume we have mood, voice, tense information passed to this function
    # Alternatively, you can modify the code to extract these from the HTML as needed

    # Since the tables are complex, we need to get mood, voice, tense from the column headers
    columns_info = []
    for col in df.columns:
        if col == 'person_number':
            columns_info.append({'column': col})
            continue
        features = {'column': col}
        # Split the column name to extract features
        tokens = col.split()
        for token in tokens:
            if token in mood_map:
                features['mood'] = mood_map[token]
            elif token in voice_map:
                features['voice'] = voice_map[token]
            elif token in tense_map:
                features['tense'] = tense_map[token]
        columns_info.append(features)

    # Iterate over the rows to extract forms
    for idx, row in df.iterrows():
        person_number = str(row.get('person_number', '')).lower()
        person = '-'
        number = '-'
        # Extract person and number from row labels
        for pn in person_map:
            if pn in person_number:
                person = person_map[pn]
                break
        for num in number_map:
            if num in person_number:
                number = number_map[num]
                break
        if person == '-' and number == '-':
            # Try splitting the person_number string
            tokens = person_number.split()
            for token in tokens:
                if token in person_map:
                    person = person_map[token]
                elif token in number_map:
                    number = number_map[token]
        # Iterate over the columns to get forms
        for col_info in columns_info:
            col = col_info['column']
            if col == 'person_number':
                continue
            form = row[col]
            if isinstance(form, str) and form.strip() != '':
                form = form.strip()
                bare_text = strip_accents(form)
                bare_base_form = strip_accents(base_word)
                mood = col_info.get('mood', '-')
                voice = col_info.get('voice', '-')
                tense = col_info.get('tense', '-')
                # Handle non-finite forms
                if person == '-' and number == '-' and 'person_number' in df.columns:
                    if 'singular' in person_number:
                        number = 's'
                    elif 'dual' in person_number:
                        number = 'd'
                    elif 'plural' in person_number:
                        number = 'p'
                    else:
                        number = '-'
                    if 'first' in person_number:
                        person = '1'
                    elif 'second' in person_number:
                        person = '2'
                    elif 'third' in person_number:
                        person = '3'
                    else:
                        person = '-'
                morph_code = generate_morph_code(
                    part_of_speech='v',
                    person=person,
                    number=number,
                    tense=tense,
                    mood=mood,
                    voice=voice,
                    gender='-',
                    case='-',
                    degree='-'
                )
                form_entry = {
                    'id': sequence_num,
                    'text': form,
                    'bare_text': bare_text,
                    'sequence_num': sequence_num,
                    'morph_code': morph_code,
                    'base_form': base_word,
                    'bare_base_form': bare_base_form,
                    'definition': definition
                }
                forms.append(form_entry)
                sequence_num += 1

    return forms

def parse_verb_table_specific(table, base_word, sequence_num_start, definition):
    """
    Parse the specific 'Present (uncontracted)' verb table for ἀάω.
    """
    forms = []
    sequence_num = sequence_num_start

    # Initialize grammatical feature mappings
    person_number_list = [
        ('1', 's'),  # First person singular
        ('2', 's'),  # Second person singular
        ('3', 's'),  # Third person singular
        ('2', 'd'),  # Second person dual
        ('3', 'd'),  # Third person dual
        ('1', 'p'),  # First person plural
        ('2', 'p'),  # Second person plural
        ('3', 'p'),  # Third person plural
    ]

    mood_map = {
        'indicative': 'i',
        'subjunctive': 's',
        'optative': 'o',
        'imperative': 'm',
        'infinitive': 'n',
        'participle': 'p'
    }

    voice_map = {
        'active': 'a',
        'middle': 'm',
        'passive': 'p',
        'mediopassive': 'e'
    }

    tense_map = {
        'present': 'p',
        # Add other tenses if needed
    }

    # Initialize current grammatical features
    current_voice = '-'
    current_mood = '-'
    current_tense = 'p'  # Present tense

    # Iterate over table rows
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all(['th', 'td'])
        if not cells:
            continue
        first_cell_text = cells[0].get_text(strip=True).lower()
        # Check for voice labels
        if first_cell_text in voice_map:
            current_voice = voice_map[first_cell_text]
            continue
        # Check for mood labels
        elif first_cell_text in mood_map:
            current_mood = mood_map[first_cell_text]
            continue
        # Check for participle gender labels
        elif first_cell_text in ['m', 'f', 'n']:
            current_gender = first_cell_text
            # Process participle forms
            if len(cells) > 1:
                form = cells[1].get_text(strip=True)
                if form and form not in ['-', '—']:
                    bare_text = strip_accents(form)
                    bare_base_form = strip_accents(base_word)
                    morph_code = generate_morph_code(
                        part_of_speech='v',
                        person='-',
                        number='-',
                        tense=current_tense,
                        mood='p',  # Participle
                        voice=current_voice,
                        gender=current_gender,
                        case='-',
                        degree='-'
                    )
                    form_entry = {
                        'id': sequence_num,
                        'text': form,
                        'bare_text': bare_text,
                        'sequence_num': sequence_num,
                        'morph_code': morph_code,
                        'base_form': base_word,
                        'bare_base_form': bare_base_form,
                        'definition': definition
                    }
                    forms.append(form_entry)
                    sequence_num += 1
            continue
        # Check for infinitive
        elif first_cell_text == 'infinitive':
            # Process infinitive form
            if len(cells) > 1:
                form = cells[1].get_text(strip=True)
                if form and form not in ['-', '—']:
                    bare_text = strip_accents(form)
                    bare_base_form = strip_accents(base_word)
                    morph_code = generate_morph_code(
                        part_of_speech='v',
                        person='-',
                        number='-',
                        tense=current_tense,
                        mood='n',  # Infinitive
                        voice=current_voice,
                        gender='-',
                        case='-',
                        degree='-'
                    )
                    form_entry = {
                        'id': sequence_num,
                        'text': form,
                        'bare_text': bare_text,
                        'sequence_num': sequence_num,
                        'morph_code': morph_code,
                        'base_form': base_word,
                        'bare_base_form': bare_base_form,
                        'definition': definition
                    }
                    forms.append(form_entry)
                    sequence_num += 1
            continue
        # Skip notes and empty rows
        elif first_cell_text in ['notes:', 'note:', '']:
            continue

        # Process rows with verb forms
        forms_in_row = [cell.get_text(strip=True) for cell in cells]
        # Remove empty forms and labels
        forms_in_row = [form for form in forms_in_row if form not in ['', '-', '—']]
        # Skip rows with insufficient data
        if len(forms_in_row) < 2:
            continue

        # For moods with person and number
        if current_mood in ['i', 's', 'o', 'm']:  # Indicative, Subjunctive, Optative, Imperative
            for idx_form, form in enumerate(forms_in_row[1:], start=0):
                if form in ['', '-', '—']:
                    continue
                if idx_form < len(person_number_list):
                    person, number = person_number_list[idx_form]
                    bare_text = strip_accents(form)
                    bare_base_form = strip_accents(base_word)
                    morph_code = generate_morph_code(
                        part_of_speech='v',
                        person=person,
                        number=number,
                        tense=current_tense,
                        mood=current_mood,
                        voice=current_voice,
                        gender='-',
                        case='-',
                        degree='-'
                    )
                    form_entry = {
                        'id': sequence_num,
                        'text': form,
                        'bare_text': bare_text,
                        'sequence_num': sequence_num,
                        'morph_code': morph_code,
                        'base_form': base_word,
                        'bare_base_form': bare_base_form,
                        'definition': definition
                    }
                    forms.append(form_entry)
                    sequence_num += 1
        else:
            # Handle other moods if necessary
            pass

    return forms


In [16]:
def strip_accents(s):
    """Remove accents from a given string."""
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

def clean_dataframe(df):
    """Clean the DataFrame by removing unwanted characters and standardizing the data."""
    df = df.dropna(how='all', axis=0)  # Remove empty rows

    # Remove unwanted characters and symbols
    df = df.replace({'➤': '', '•': '', '\[': '', '\]': '', '\{': '', '\}': '', '—': ' '}, regex=True)
    df = df.replace(r"\([^)]*\)", "", regex=True)  # Remove content within parentheses

    # Make all strings lowercase
    df = df.map(lambda s: s.lower() if isinstance(s, str) else s)

    # Split strings to remove extraneous information
    df = df.map(lambda s: s.split(',', 1)[0] if isinstance(s, str) else s)
    df = df.map(lambda s: s.split('-', 1)[0] if isinstance(s, str) else s)
    df = df.map(lambda s: s.split('+', 1)[0] if isinstance(s, str) else s)
    df = df.map(lambda s: s.split('/', 1)[0] if isinstance(s, str) else s)
    df = df.map(lambda s: s.split('\\', 1)[0] if isinstance(s, str) else s)

    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

    df = df.replace(r"\s\s+", " ", regex=True)  # Replace multiple spaces with a single space

    return df

def generate_variations(word, language='ancient greek', show=False, definition_map=None):
    # Fetch the Wiktionary page
    url = requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
    soup = bs(url.text, 'html.parser')

    # Remove unwanted elements
    for tag in soup.find_all('span', {'class': 'tr Latn'}):
        tag.decompose()
    for tag in soup.find_all('table', {'class': 'audiotable'}):
        tag.decompose()
    for tag in soup.find_all('sup'):
        tag.decompose()

    # Find all tables
    tables = soup.find_all('table')
    gender = None

    # DataFrames
    dataframes = pd.read_html(str(soup))

    # Get the definition of the base word from the definition_map
    if definition_map is not None:
        normalized_word = strip_accents(word).lower()
        definition = definition_map.get(normalized_word, '')
    else:
        definition = ''
        
    # Initialize list to collect forms
    forms = []
    sequence_num = 1  # To keep track of sequence numbers

    for i, df in enumerate(dataframes):
        # Find the header corresponding to the table
        header = tables[i].find_previous('h2').text.replace('[edit]', '').lower().strip()

        # Check if the language matches
        if header != language:
            continue

        # Get the part of speech
        if tables[i].find_previous('h3'):
            part_of_speech = tables[i].find_previous('h3').text.replace('[edit]', '').replace('\"', '').lower().strip()
        else:
            part_of_speech = ''

        # Get gender
        gender_tag = tables[i].find_previous(class_="gender")
        if gender_tag:
            gender = gender_tag.find_next("abbr").text.lower()
            if 'm' in gender:
                gender = 'm'
            elif 'f' in gender:
                gender = 'f'
            elif 'n' in gender:
                gender = 'n'
            else:
                gender = '-'
        else:
            gender = '-'

        # Clean the DataFrame
        df = clean_dataframe(df)

        # Process based on part of speech
        if part_of_speech == 'noun':
            forms.extend(parse_noun_table(df, word, gender, sequence_num, definition))
            sequence_num += len(df.index)
        elif part_of_speech == 'adjective':
            forms.extend(parse_adjective_table(df, word, sequence_num, definition))
            sequence_num += len(df.index)
        elif part_of_speech == 'verb':
            forms.extend(parse_verb_table(df, word, sequence_num, definition))
            sequence_num += len(df.index)

    return pd.DataFrame(forms)

In [17]:
def update_csv_with_wiktionary(csv_path, base_word):
    """
    Update the CSV file with forms from Wiktionary by comparing existing morph codes
    and adding any missing forms.
    """
    # Read the original CSV
    df = pd.read_csv(csv_path)
    
    # Get variations from Wiktionary
    scraped_df = generate_variations(base_word)
    
    if scraped_df.empty:
        print(f"No new forms found for {base_word}.")
        return csv_path  # Return the original path if no updates
    
    # Ensure morph_code is a string
    df['morph_code'] = df['morph_code'].astype(str)
    scraped_df['morph_code'] = scraped_df['morph_code'].astype(str)
    
    # Filter rows corresponding to the base word
    base_word_block = df[df['base_form'] == base_word]
    existing_morph_codes = set(base_word_block['morph_code'])
    
    # Get new forms not in the original CSV
    new_forms = scraped_df[~scraped_df['morph_code'].isin(existing_morph_codes)]
    
    if new_forms.empty:
        print(f"All forms of {base_word} are already present in the dataset.")
        return csv_path  # Return the original path if no new forms to add
    
    # Append new forms to the DataFrame
    updated_df = pd.concat([df, new_forms], ignore_index=True)
    
    # Save the updated DataFrame to a new CSV
    updated_csv_path = csv_path.replace('.csv', '_updated.csv')
    updated_df.to_csv(updated_csv_path, index=False)
    
    print(f"Updated CSV saved to {updated_csv_path}")
    return updated_csv_path

In [18]:
# Read the CSV file
greek_words_df = pd.read_csv('greek_words.csv', encoding='utf-8', sep='\t', index_col=False)

# Apply normalization to the base forms
greek_words_df['normalized_base_form'] = greek_words_df['base_form'].apply(normalize_word)

# Create a mapping from normalized_base_form to definition
definition_map = greek_words_df.set_index('normalized_base_form')['definition'].to_dict()

# Define desired columns in the correct order
desired_columns = ['id', 'text', 'bare_text', 'sequence_num', 'morph_code', 'base_form', 'bare_base_form', 'definition']

# Initialize an empty list to collect all forms
all_forms = []

# Get the list of unique base forms
base_forms = greek_words_df['base_form'].unique()
test_base_forms = base_forms[:5]
# Iterate over each base_form
for base_form in test_base_forms:
    print(f"Processing word: {base_form}")
    try:
        # Generate variations
        variations_df = generate_variations(base_form, definition_map=definition_map)
        
        if not variations_df.empty:
            # Ensure columns are in the correct order and only include desired columns
            variations_df = variations_df[desired_columns]
            # Add to all_forms
            all_forms.append(variations_df)
        else:
            print(f"No variations found for {base_form}. Copying data from greek_words.csv")
            # Get data from greek_words_df where base_form == base_form
            word_data = greek_words_df[greek_words_df['base_form'] == base_form]
            # Ensure columns are in the correct order and only include desired columns
            word_data = word_data[desired_columns]
            all_forms.append(word_data)
    except Exception as e:
        print(f"Error processing {base_form}: {e}")
        # Get data from greek_words_df where base_form == base_form
        word_data = greek_words_df[greek_words_df['base_form'] == base_form]
        # Ensure columns are in the correct order and only include desired columns
        word_data = word_data[desired_columns]
        all_forms.append(word_data)

# Concatenate all forms into a single DataFrame
all_forms_df = pd.concat(all_forms, ignore_index=True)

# Reset index to avoid Unnamed columns
all_forms_df.reset_index(drop=True, inplace=True)

# Save to a new CSV file with tab separator
all_forms_df.to_csv('greek_words_updated.csv', sep='\t', index=False, encoding='utf-8')
print("All forms have been successfully saved to 'greek_words_updated.csv'.")

Processing word: ἀάατος


C:\Users\Jacob\AppData\Local\Temp\ipykernel_24028\147961841.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = pd.read_html(str(soup))


No variations found for ἀάατος. Copying data from greek_words.csv
Processing word: ἀαγής


C:\Users\Jacob\AppData\Local\Temp\ipykernel_24028\147961841.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = pd.read_html(str(soup))


No variations found for ἀαγής. Copying data from greek_words.csv
Processing word: ἄαπτος


C:\Users\Jacob\AppData\Local\Temp\ipykernel_24028\147961841.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = pd.read_html(str(soup))


No variations found for ἄαπτος. Copying data from greek_words.csv
Processing word: ἄατος


C:\Users\Jacob\AppData\Local\Temp\ipykernel_24028\147961841.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = pd.read_html(str(soup))


Error processing ἄατος: No tables found
Processing word: ἀάω
All forms have been successfully saved to 'greek_words_updated.csv'.


C:\Users\Jacob\AppData\Local\Temp\ipykernel_24028\147961841.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = pd.read_html(str(soup))
